In [ ]:
! pip install wikipedia-api
import wikipediaapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from IPython.testing import test
sports_titles = ["Football", "Basketball", "Tennis", "Swimming", "Golf"]
politics_titles = ["Government", "Elections", "Political_parties", "Democracy", "Dictatorship"]
test_titles = ["volley_ball","Politics"]
wiki = wikipediaapi.Wikipedia('en')
documents = []
documents2 = []
test_doc = []
for title in sports_titles:
    page = wiki.page(title)
    documents.append(page.text)

for title in politics_titles :
    page2 = wiki.page(title)
    documents2.append(page2.text)

for title in test_titles:
    page3 = wiki.page(title)
    test_doc.append(page3.text)

In [ ]:
len(documents[4])

52562

In [ ]:
def class_prob(doc_labels):

    class_counts = {}
    total_docs = len(doc_labels)

    # Count the number of documents for each class
    for label in doc_labels:
        if label in class_counts:
            class_counts[label] += 1
        else:
            class_counts[label] = 1

    # Calculate the probability of each class
    class_probabilities = {}
    for label, count in class_counts.items():
        class_probabilities[label] = count / total_docs

    return class_probabilities


In [ ]:
l = ['moaz','zaki','moaz','zaki']
class_prob(l)

{'moaz': 0.5, 'zaki': 0.5}

In [ ]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

nltk.download('punkt')
nltk.download('stopwords')

def preprocessing(doc):
    # Convert to lowercase
    doc = doc.lower()

    # Remove all non-letter characters
    doc = re.sub(r'[^a-z\s]', '', doc)

    # Tokenize
    tokens = word_tokenize(doc)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Join tokens back into a string
    preprocessed_doc = ' '.join(tokens)

    return preprocessed_doc


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
doc11='Lw msh bt3ml lemmatization aw stemming blash ahsn w shof haga from scratch adman'
preprocessing(doc11)

'lw msh btml lemmat aw stem blash ahsn w shof haga scratch adman'

In [ ]:
def getTokens(doc):
    """
    Extracts the unique tokens (words) from a preprocessed document.

    Parameters:
    doc (str): A preprocessed document string.

    Returns:
    set: A set of unique tokens (words) in the document.
    """
    tokens = doc.split()
    return set(tokens)


In [ ]:
def conditional_prob(docs, labels):
    # Get all unique class labels
    class_labels = set(labels)

    # Get all unique tokens in the entire corpus
    all_tokens = set([token for doc in docs for token in  getTokens(doc)])

    # Initialize an empty dictionary to store conditional probabilities
    conditional_probs = {}

    # Loop over all tokens
    for token in all_tokens:
        # Loop over all class labels
        for class_label in class_labels:
            # Get all documents in the class that contain the token
            class_docs = [doc for doc, label in zip(docs, labels) if label == class_label and token in  getTokens(doc)]

            # Calculate the frequency of the token in the class
            token_freq = sum([doc.count(token) for doc in class_docs])

            # Calculate the total number of tokens in the class
            total_tokens = sum([len( getTokens(doc)) for doc, label in zip(docs, labels) if label == class_label])

            # Calculate the conditional probability of the token given the class
            alpha = 1 # Laplace smoothing parameter
            conditional_prob = (token_freq + alpha) / (total_tokens + alpha * len(all_tokens))
            conditional_probs[(class_label, token)] = conditional_prob

    return conditional_probs


In [ ]:
docs = documents+ documents2
labels = ["sports","sports","sports","sports","sports","politics","politics","politics","politics","politics"]

In [ ]:
d = conditional_prob(docs , labels)

In [ ]:
d

{('politics', 'drowning.Adverse'): 4.0494027130998175e-05,
 ('sports', 'drowning.Adverse'): 7.14643035803616e-05,
 ('politics', 'principal'): 8.098805426199635e-05,
 ('sports', 'principal'): 0.00010719645537054242,
 ('politics', 'Bryant,'): 4.0494027130998175e-05,
 ('sports', 'Bryant,'): 7.14643035803616e-05,
 ('politics', 'election).In'): 8.098805426199635e-05,
 ('sports', 'election).In'): 3.57321517901808e-05,
 ('politics', 'nativism,'): 8.098805426199635e-05,
 ('sports', 'nativism,'): 3.57321517901808e-05,
 ('politics', 'Cedar'): 8.098805426199635e-05,
 ('sports', 'Cedar'): 3.57321517901808e-05,
 ('politics', 'inherent'): 8.098805426199635e-05,
 ('sports', 'inherent'): 3.57321517901808e-05,
 ('politics', 'cord'): 4.0494027130998175e-05,
 ('sports', 'cord'): 0.00042878582148216966,
 ('politics', 'eroded'): 4.0494027130998175e-05,
 ('sports', 'eroded'): 7.14643035803616e-05,
 ('politics', 'Based'): 4.0494027130998175e-05,
 ('sports', 'Based'): 0.00021439291074108483,
 ('politics', 're

In [ ]:
d[ ('sports', 'football')]

0.010540984778103337

In [ ]:
d[('politics', 'football')]

4.0494027130998175e-05

In [ ]:
# Preprocess the documents
preprocessed_docs = [preprocessing(doc) for doc in docs]

# Calculate the class probabilities
class_probs = class_prob(labels)

# Calculate the conditional probabilities
cond_probs = conditional_prob(preprocessed_docs, labels)

# Save the probabilities to hard disk
import pickle
with open('probabilities.pkl', 'wb') as f:
    pickle.dump((class_probs, cond_probs), f)


In [ ]:
import numpy as np

In [ ]:
def predict(test_doc):
    # Load the probabilities from hard disk
    with open('probabilities.pkl', 'rb') as f:
        class_probs, cond_probs = pickle.load(f)

    # Preprocess the test document
    preprocessed_test_doc = preprocessing(test_doc)

    # Get the set of unique tokens in the test document
    test_tokens = getTokens(preprocessed_test_doc)

    # Initialize a dictionary to store the log probabilities of the test document for each class
    log_probs = {}

    # Loop over all class labels
    for class_label in class_probs.keys():
        # Calculate the log probability of the class given the test document
        log_prob = 0
        for token in test_tokens:
            if (class_label, token) in cond_probs:
                log_prob += np.log(cond_probs[(class_label, token)])
        log_prob += np.log(class_probs[class_label])
        log_probs[class_label] = log_prob

    # Return the class with the highest log probability
    return max(log_probs, key=log_probs.get)


In [ ]:
test_doc[0]

'Volleyball is a team sport in which two teams of six players are separated by a net. Each team tries to score points by grounding a ball on the other team\'s court under organized rules. It has been a part of the official program of the Summer Olympic Games since Tokyo 1964. Beach volleyball was introduced to the programme at the Atlanta 1996 Summer Olympics. The adapted version of volleyball at the Summer Paralympic Games is sitting volleyball.\nThe complete set of rules is extensive, but play essentially proceeds as follows: a player on one of the teams begins a \'rally\' by serving the ball (tossing or releasing it and then hitting it with a hand or arm), from behind the back boundary line of the court, over the net, and into the receiving team\'s court. The receiving team must not let the ball be grounded within their court. The team may touch the ball up to three times to return the ball to the other side of the court, but individual players may not touch the ball twice consecuti

In [ ]:
test_doc[1]

'Politics (from Greek: Πολιτικά, politiká, \'affairs of the cities\') is the set of activities that are associated with making decisions in groups, or other forms of power relations among individuals, such as the distribution of resources or status. The branch of social science that studies politics and government is referred to as political science.\nIt may be used positively in the context of a "political solution" which is compromising and nonviolent, or descriptively as "the art or science of government", but also often carries a negative connotation. The concept has been defined in various ways, and different approaches have fundamentally differing views on whether it should be used extensively or limitedly, empirically or normatively, and on whether conflict or co-operation is more essential to it.\nA variety of methods are deployed in politics, which include promoting one\'s own political views among people, negotiation with other political subjects, making laws, and exercising 

In [ ]:
predict(test_doc[0])

'sports'

In [ ]:
predict(test_doc[1])

'politics'